In [1]:
from torch import optim
from lib.datasets import get_stock_price,sample_indices,train_test_split
from lib.aug import apply_augmentations,parse_augmentations
import torch
from torch import nn
import torch.nn.functional as F
from typing import List
from lib.utils import sample_indices


In [2]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
    ],
    "device" : "cuda:0",
    "depth" : 4,
}

In [3]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))
print("Before augmentation shape:",x_real_train.shape)
if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_sig = apply_augmentations(x_real_train,sig_config["augmentations"])
    # Input dimension of encoder
    # We'll flat the tensor
    input_dim = x_aug_sig.shape[1]*x_aug_sig.shape[2]
print("After augmentation shape:",x_aug_sig.shape)
x_aug_sig = x_aug_sig.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
Before augmentation shape: torch.Size([985, 20, 1])
torch.Size([985, 20, 2])
torch.Size([985, 39, 4])
After augmentation shape: torch.Size([985, 39, 4])


In [ ]:
from models.cvae import CVAE,CAVE_train

# Model hyperparameters
lr = 1e-3  
batch_size = 128
epoch = 20001
latent_dim = 20  
hidden_dims = [156, 100, latent_dim]  

cvae = CVAE(
    x_aug_sig=x_aug_sig,
    x_original=x_real_train.to("cuda"),  
    epoch=epoch,
    batch_size=batch_size,
    hidden_dims=hidden_dims,
    latent_dim=latent_dim,
    device='cuda'
)
optimizer = torch.optim.Adam(cvae.parameters(), lr=lr)

# Train the model
CAVE_train(cvae, optimizer)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper___cat)